In [2]:
import torch
import torchvision
from torch2trt import torch2trt
import time

# create some regular pytorch model...
model = torchvision.models.resnet50(pretrained=True).cuda().half().eval()

# create example data
x = torch.ones((1, 3, 224, 224)).cuda().half()

# convert to TensorRT feeding sample data as input
model_trt = torch2trt(model, [x], fp16_mode=True)

In [3]:
time_model_s = time.time()
y = model(x)
time_model_e = time.time()
y_trt = model_trt(x)
time_model_trt_e = time.time()

print(time_model_e - time_model_s, time_model_trt_e - time_model_e)

# check the output against PyTorch
print(torch.max(torch.abs(y - y_trt)))

0.008980274200439453 0.0010447502136230469
tensor(0.0107, device='cuda:0', dtype=torch.float16, grad_fn=<MaxBackward1>)


In [5]:
torch.save(model_trt.state_dict(), 'resnet50_trt.pth')

In [6]:
from torch2trt import TRTModule

model_trt = TRTModule()

model_trt.load_state_dict(torch.load('resnet50_trt.pth'))

<All keys matched successfully>